# 다중 레이블 분류

In [ ]:
# MNIST 데이터셋 사용
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

# 샘플이 하나의 행, 특성이 하나의 열로 구성된 배열을 가진 data 키 
# 레이블 배열을 담은 target 키
X, y = mnist["data"], mnist["target"]
X.shape

# 이미지가 70,000개 있고 각 이미지에는 784개의 특성 (28x28)
# 개개의 특성 : 0(흰색) ~ 255(검은색) 픽셀 강도
# (70,000, 784)


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

some_digit = X[0]
some_digit_image = some_digit.reshape(28,28)

plt.imshow(some_digit_image, cmap="binary")
plt.axis("off")
plt.show()


In [ ]:
y = y.astype(np.uint8)

# 훈련세트 : 앞쪽 6,000개 이미지
# 테스트세트 : 뒤쪽 10,000개 이미지 
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
# 다중 레이블 분류

from sklearn.neighbors import KNeighborsClassifier

y_train_large = (y_train >=7)    # 큰 값 (7,8,9)
y_train_odd = (y_train % 2 == 1) # 홀수
y_multilabel = np.c_[y_train_large, y_train_odd]
# np.c_: 1차원 배열을 세로로 만들어 2차원 배열로 합침

# 학습
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, y_multilabel)

# 예측
knn_clf.predict([some_digit])

In [ ]:
# 다중 레이블 분류기 평가 방법
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

# 모든 레이블에 대한 F1 점수의 평균 계산
y_train_knn_pred = cross_val_predict(knn_clf, X_train, y_multilabel, cv=3)
f1_score(y_multilabel, y_train_knn_pred, average="macro")

# average="macro": 모든 레이블의 가중치가 같다고 가정
# average="weighted": 레이블에 클래스의 지지도(support, 타깃 레이블에 속한 샘플 수)를 가중치로 부여
# average="micro": 모든 클래스의 FP,FN,TP 총합을 이용해 F1 계산

# 0.976410265560605

In [ ]:
# 다중 출력 분류 (Multioutput-Multiclass Classification)
# 사례 : 노이즈가 있는 숫자 이미지를 입력으로 받고, MNIST 이미지처럼 픽셀의 강도를 담은 배열 출력
# 레이블 : 744개의 픽셀, 레이블의 값 (0~255 픽셀강도)

# 입력 데이터에 노이즈 추가
noise = np.random.randint(0, 100, (len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100, (len(X_test), 784))
X_test_mod = X_test + noise

# 출력 데이터는 원본 이미지
y_train_mod = X_train
y_test_mod = X_test

In [ ]:
some_index = 0

In [ ]:
# plot_digit() 함수
def plot_digit(data): 
    image = data.reshape(28, 28)
    plt.imshow(image, cmap=mpl.cm.binary, interpolation="nearest")
    plt.axis("off")

In [ ]:
knn_clf.fit(X_train_mod, y_train_mod)
clean_digit = knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)